In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('data/NVDA_30min_sample.csv')

In [3]:
data

,timestamp,open,high,low,close,volume
0,2024-02-26 04:00:00,790.90,798.00,787.87,797.98,27109
1,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848
2,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654
3,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711
4,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169
...,...,...,...,...,...,...
347,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844
348,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907
349,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558
350,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359


In [4]:
data['ret']=data['close'].pct_change()
data

,timestamp,open,high,low,close,volume,ret
0,2024-02-26 04:00:00,790.90,798.00,787.87,797.98,27109,NaN
1,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848,0.003158
2,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654,-0.002249
3,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711,-0.001840
4,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169,0.000602
...,...,...,...,...,...,...,...
347,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844,-0.000443
348,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907,0.010885
349,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558,0.000058
350,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359,0.002285


In [5]:
data=data.dropna()

In [6]:
wq_alphas=data.rename(columns={
    'open': 'S_DQ_OPEN', 
    'high': 'S_DQ_HIGH', 
    'low': 'S_DQ_LOW', 
    'close': 'S_DQ_CLOSE', 
    'volume': 'S_DQ_VOLUME', 
    'ret': 'S_DQ_PCTCHANGE',
})
wq_alphas=wq_alphas[['timestamp','S_DQ_OPEN','S_DQ_HIGH','S_DQ_LOW','S_DQ_CLOSE','S_DQ_VOLUME','S_DQ_PCTCHANGE']]
wq_alphas

,timestamp,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE
1,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848,0.003158
2,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654,-0.002249
3,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711,-0.001840
4,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169,0.000602
5,2024-02-26 06:30:00,797.64,797.82,795.70,796.00,8632,-0.002144
...,...,...,...,...,...,...,...
347,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844,-0.000443
348,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907,0.010885
349,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558,0.000058
350,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359,0.002285


In [7]:
# Add a date column to group by
wq_alphas['timestamp'] = pd.to_datetime(wq_alphas['timestamp'])

wq_alphas['date'] = wq_alphas['timestamp'].dt.date

# Calculate the typical price
wq_alphas['Typical_Price'] = (wq_alphas['S_DQ_HIGH'] + wq_alphas['S_DQ_LOW'] + wq_alphas['S_DQ_CLOSE']) / 3

# Calculate the product of the typical price and volume
wq_alphas['TP_Volume'] = wq_alphas['Typical_Price'] * wq_alphas['S_DQ_VOLUME']

# Group by the date column, and then calculate the sum of TP_Volume and sum of volume for each group
grouped = wq_alphas.groupby('date').apply(lambda x: x.assign(vwap=x['TP_Volume'].cumsum() / x['S_DQ_VOLUME'].cumsum()))

# Drop the intermediate columns if you no longer need them
grouped = grouped.drop(['Typical_Price', 'TP_Volume','date'], axis=1)

/var/folders/bm/bm_nkxfx3w1303dr6gyz9cg00000gn/T/ipykernel_35520/2039745111.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = wq_alphas.groupby('date').apply(lambda x: x.assign(vwap=x['TP_Volume'].cumsum() / x['S_DQ_VOLUME'].cumsum()))


In [8]:
grouped

timestamp  S_DQ_OPEN  S_DQ_HIGH  S_DQ_LOW  \
date                                                                 
2024-02-26 1   2024-02-26 04:30:00     798.01     801.00    797.01   
           2   2024-02-26 05:00:00     800.07     800.07    797.30   
           3   2024-02-26 05:30:00     798.63     798.63    797.12   
           4   2024-02-26 06:00:00     797.20     797.97    796.61   
           5   2024-02-26 06:30:00     797.64     797.82    795.70   
...                            ...        ...        ...       ...   
2024-03-11 347 2024-03-11 17:30:00     858.00     858.90    856.52   
           348 2024-03-11 18:00:00     857.34     866.45    857.34   
           349 2024-03-11 18:30:00     866.50     867.80    863.93   
           350 2024-03-11 19:00:00     866.76     870.04    865.04   
           351 2024-03-11 19:30:00     868.40     869.73    867.50   

                S_DQ_CLOSE  S_DQ_VOLUME  S_DQ_PCTCHANGE        vwap  
date                                                                 
2024-02-26 1        800.50        37848        0.003158  799.503333  
           2        798.70         8654       -0.002249  799.351972  
           3        797.23         7711       -0.001840  799.111314  
           4        797.71         6169        0.000602  798.939541  
           5        796.00         8632       -0.002144  798.635246  
...                    ...          ...             ...         ...  
2024-03-11 347      857.12        22844       -0.000443  865.791689  
           348      866.45       114907        0.010885  865.785653  
           349      866.50        67558        0.000058  865.786087  
           350      868.48        90359        0.002285  865.790198  
           351      869.73        43744        0.001439  865.793273  

[351 rows x 8 columns]

In [9]:
wq_alphas= grouped.reset_index()  # This only drops the index, not the column
wq_alphas

,date,level_1,timestamp,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE,vwap
0,2024-02-26,1,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848,0.003158,799.503333
1,2024-02-26,2,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654,-0.002249,799.351972
2,2024-02-26,3,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711,-0.001840,799.111314
3,2024-02-26,4,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169,0.000602,798.939541
4,2024-02-26,5,2024-02-26 06:30:00,797.64,797.82,795.70,796.00,8632,-0.002144,798.635246
...,...,...,...,...,...,...,...,...,...,...
346,2024-03-11,347,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844,-0.000443,865.791689
347,2024-03-11,348,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907,0.010885,865.785653
348,2024-03-11,349,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558,0.000058,865.786087
349,2024-03-11,350,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359,0.002285,865.790198


In [10]:
wq_alphas=wq_alphas.drop(['date','level_1'],axis=1)
wq_alphas

,timestamp,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE,vwap
0,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848,0.003158,799.503333
1,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654,-0.002249,799.351972
2,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711,-0.001840,799.111314
3,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169,0.000602,798.939541
4,2024-02-26 06:30:00,797.64,797.82,795.70,796.00,8632,-0.002144,798.635246
...,...,...,...,...,...,...,...,...
346,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844,-0.000443,865.791689
347,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907,0.010885,865.785653
348,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558,0.000058,865.786087
349,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359,0.002285,865.790198


In [11]:
import sys 
sys.path.append('utils')
from wq_alpha_generator import get_alpha

NVDA_alphas=get_alpha(wq_alphas)

In [12]:
NVDA_alphas

,timestamp,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE,vwap,alpha001,alpha017,alpha023,alpha030,alpha034
0,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848,0.003158,799.503333,NaN,NaN,0.00,NaN,NaN
1,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654,-0.002249,799.351972,NaN,NaN,0.00,NaN,NaN
2,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711,-0.001840,799.111314,NaN,NaN,0.00,NaN,NaN
3,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169,0.000602,798.939541,NaN,NaN,0.00,NaN,NaN
4,2024-02-26 06:30:00,797.64,797.82,795.70,796.00,8632,-0.002144,798.635246,NaN,NaN,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844,-0.000443,865.791689,0.219512,-0.005572,0.00,0.141877,0.915663
347,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907,0.010885,865.785653,0.821646,-0.489867,0.00,0.067430,0.045181
348,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558,0.000058,865.786087,0.821646,-0.105548,0.00,0.003455,0.262048
349,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359,0.002285,865.790198,0.821646,-0.375501,0.00,0.000669,0.599398


In [13]:
NVDA_alphas=NVDA_alphas.dropna()
NVDA_alphas

,timestamp,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE,vwap,alpha001,alpha017,alpha023,alpha030,alpha034
28,2024-02-26 18:30:00,788.3953,789.88,788.05,789.40,33345,0.001713,795.370806,0.821646,-0.226065,0.00,0.000585,0.629518
29,2024-02-26 19:00:00,789.4000,789.60,787.60,788.02,19892,-0.001748,795.366996,0.545732,-0.015627,0.00,0.002266,0.457831
30,2024-02-26 19:30:00,788.3000,789.50,788.15,789.28,19814,0.001599,795.363515,0.371951,-0.263641,0.00,0.002310,0.081325
31,2024-02-27 04:00:00,793.0000,794.65,791.48,793.55,15733,0.005410,793.226667,0.821646,-0.143330,0.00,0.002251,0.253012
32,2024-02-27 04:30:00,793.0200,794.24,792.10,793.94,6141,0.000491,793.282816,0.821646,-0.069196,0.00,0.000382,0.271084
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2024-03-11 17:30:00,858.0000,858.90,856.52,857.12,22844,-0.000443,865.791689,0.219512,-0.005572,0.00,0.141877,0.915663
347,2024-03-11 18:00:00,857.3400,866.45,857.34,866.45,114907,0.010885,865.785653,0.821646,-0.489867,0.00,0.067430,0.045181
348,2024-03-11 18:30:00,866.5000,867.80,863.93,866.50,67558,0.000058,865.786087,0.821646,-0.105548,0.00,0.003455,0.262048
349,2024-03-11 19:00:00,866.7600,870.04,865.04,868.48,90359,0.002285,865.790198,0.821646,-0.375501,0.00,0.000669,0.599398


In [18]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier,Pool,metrics,cv

# Assuming NVDA_alphas is your DataFrame
NVDA_alphas['Return_Direction'] = (NVDA_alphas['S_DQ_PCTCHANGE'] > 0).astype(int)

# Define the features and the target
features = ['alpha001', 'alpha017', 'alpha023', 'alpha030', 'alpha034']
target = 'Return_Direction'

# Split the data into training and testing sets
train_df, test_df = train_test_split(NVDA_alphas, test_size=0.3, random_state=42, shuffle=False)

# Apply the shift to avoid lookahead bias
X_train = train_df[features].shift(-1)
y_train = train_df[target]
X_test = test_df[features].shift(-1)
y_test = test_df[target]

# Drop the NaN values created by the shift
X_train.dropna(inplace=True)
X_test.dropna(inplace=True)

# Align y_train and y_test by dropping the first row to match the X after shift
y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

# clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf=CatBoostClassifier(random_seed=42)
# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
predictions = clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.2f}')


/var/folders/bm/bm_nkxfx3w1303dr6gyz9cg00000gn/T/ipykernel_35520/1133376811.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NVDA_alphas['Return_Direction'] = (NVDA_alphas['S_DQ_PCTCHANGE'] > 0).astype(int)


Learning rate set to 0.005449
0:	learn: 0.6928352	total: 391us	remaining: 391ms
1:	learn: 0.6924910	total: 645us	remaining: 322ms
2:	learn: 0.6921686	total: 897us	remaining: 298ms
3:	learn: 0.6919580	total: 1.13ms	remaining: 282ms
4:	learn: 0.6916666	total: 1.37ms	remaining: 273ms
5:	learn: 0.6912965	total: 1.61ms	remaining: 267ms
6:	learn: 0.6909742	total: 1.85ms	remaining: 263ms
7:	learn: 0.6906773	total: 2.09ms	remaining: 259ms
8:	learn: 0.6902773	total: 2.33ms	remaining: 256ms
9:	learn: 0.6898955	total: 2.56ms	remaining: 254ms
10:	learn: 0.6895330	total: 2.81ms	remaining: 252ms
11:	learn: 0.6892374	total: 3.04ms	remaining: 251ms
12:	learn: 0.6889042	total: 3.28ms	remaining: 249ms
13:	learn: 0.6887057	total: 3.52ms	remaining: 248ms
14:	learn: 0.6885315	total: 3.75ms	remaining: 246ms
15:	learn: 0.6882182	total: 3.99ms	remaining: 245ms
16:	learn: 0.6879302	total: 4.22ms	remaining: 244ms
17:	learn: 0.6874829	total: 4.46ms	remaining: 243ms
18:	learn: 0.6872022	total: 4.7ms	remaining: 24

In [87]:
test_df

,timestamp,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE,vwap,alpha001,alpha017,alpha023,alpha030,alpha034,Return_Direction
254,2024-03-06 19:30:00,886.90,886.90,885.00,885.10,53256,-0.002153,884.778805,0.219512,-0.076175,0.00,0.004839,0.789157,0
255,2024-03-07 04:00:00,889.17,896.50,887.84,896.50,25289,0.012880,893.613333,0.821646,-0.416073,-6.50,0.002699,0.021084,1
256,2024-03-07 04:30:00,896.50,901.00,896.20,901.00,41631,0.005020,897.213225,0.821646,-0.137960,-14.10,0.000620,0.313253,1
257,2024-03-07 05:00:00,901.22,901.64,899.00,900.00,18686,-0.001110,897.868086,0.545732,-0.011378,-5.14,0.003567,0.888554,0
258,2024-03-07 05:30:00,899.53,903.49,899.53,903.20,19691,0.003556,898.654485,0.821646,-0.491028,-2.49,0.003331,0.216867,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844,-0.000443,865.791689,0.219512,-0.005572,0.00,0.141877,0.915663,0
347,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907,0.010885,865.785653,0.821646,-0.489867,0.00,0.067430,0.045181,1
348,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558,0.000058,865.786087,0.821646,-0.105548,0.00,0.003455,0.262048,1
349,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359,0.002285,865.790198,0.821646,-0.375501,0.00,0.000669,0.599398,1


In [88]:
# Drop the last row of test_df to match the length of predictions
test_df_adjusted = test_df.iloc[:-1].copy()

# Now you can add the predictions to the adjusted test_df
test_df_adjusted['Predictions'] = predictions
# Assuming `test_df` has columns for open ('S_DQ_OPEN') and close ('S_DQ_CLOSE') prices
# and `predictions` contains the model's predictions for the test set



# Calculate the profit/loss for each prediction
# Assuming a long position: Buy at open, sell at close
test_df['PnL'] = test_df_adjusted.apply(lambda row: row['S_DQ_CLOSE'] - row['S_DQ_OPEN'] if row['Predictions'] == 1 else 0, axis=1)

# Aggregate PnL to daily level by summing up all PnLs for each day
# Assuming 'timestamp' is in datetime format; if not, convert it using pd.to_datetime()
test_df['Date'] = test_df['timestamp'].dt.date
daily_pnl = test_df.groupby('Date')['PnL'].sum()

# Calculate the Average Daily Profit
average_daily_profit = daily_pnl.mean()

print(f"Average Daily Profit: {average_daily_profit}")


Average Daily Profit: 41.24555000000018


In [89]:
NVDA_alphas['Return_Direction'].value_counts()

Return_Direction
1    174
0    149
Name: count, dtype: int64